In [ ]:
from google.colab import files
file = files.upload()

Saving dev-access.csv to dev-access.csv


In [ ]:
import pandas as pd
df= pd.read_csv('dev-access.csv',engine='python', quotechar='|', header=None)


In [ ]:
def analyze():
    dataframe = df.copy()
    count_frame = dataframe.groupby([1]).count()
    print(count_frame)
    total_req = count_frame[0][0] + count_frame[0][1]
    num_malicious = count_frame[0][1]
    print("Total request = " , total_req)
    print("Number of Malicious requests = " , num_malicious)
    print("Percentage of Malicious request logs in dataset: {:0.2f}%".format(float(num_malicious) / total_req * 100))

In [ ]:
import sys
import os
import json
import pandas
import numpy
import optparse
import numpy as np
import keras
from keras.callbacks import TensorBoard
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout ,SimpleRNN , GRU
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from collections import OrderedDict


In [ ]:
analyze()
# df.head()

       0
1       
0  13413
1  13360
Total request =  26773
Number of Malicious requests =  13360
Percentage of Malicious request logs in dataset: 49.90%


In [ ]:
def preprocess():
    dataframe = df.copy()
    dataset = dataframe.sample(frac=1).values

    # Preprocess dataset
    X = np.asarray(dataset[:,0])
    Y = np.asarray(dataset[:,1])

    for index, item in enumerate(X):
        # Quick hack to space out json elements
        reqJson = json.loads(item, object_pairs_hook=OrderedDict)
        del reqJson['timestamp']
        del reqJson['headers']
        del reqJson['source']
        del reqJson['route']
        del reqJson['responsePayload']
        X[index] = json.dumps(reqJson, separators=(',', ':'))

    tokenizer = Tokenizer(filters='\t\n', char_level=True)
    tokenizer.fit_on_texts(X)

    # Extract and save word dictionary
    word_dict_file = 'build/word-dictionary.json'

    if not os.path.exists(os.path.dirname(word_dict_file)):
        os.makedirs(os.path.dirname(word_dict_file))

    with open(word_dict_file, 'w') as outfile:
        json.dump(tokenizer.word_index, outfile, ensure_ascii=False)

    num_words = len(tokenizer.word_index)+1
    X = tokenizer.texts_to_sequences(X)

    max_log_length = 1024
    train_size = int(len(dataset) * .75)

    X_processed = sequence.pad_sequences(X, maxlen=max_log_length)
    X_train, X_test = np.asarray(X_processed[0:train_size]), np.asarray(X_processed[train_size:len(X_processed)])
    Y_train, Y_test = np.asarray(Y[0:train_size]), np.asarray(Y[train_size:len(Y)])
    # X_train = np.expand_dims(X_train, -1)
    # Y_train = np.expand_dims(Y_train, -1)
    X_train=np.asarray(X_train).astype(np.float32)
    Y_train = np.asarray(Y_train).astype(np.float32)
    X_test = np.asarray(X_test).astype(np.float32)
    Y_test = np.asarray(Y_test).astype(np.float32)
    return X_train, X_test , Y_train, Y_test , num_words , max_log_length



In [ ]:
X_train, X_test , Y_train, Y_test,num_words , max_log_length = preprocess()

In [ ]:
!pip install scikit-plot

In [ ]:
from sklearn.metrics import confusion_matrix, recall_score ,precision_score , fbeta_score ,cohen_kappa_score  ,log_loss , roc_auc_score
from scikitplot.metrics import plot_roc
import matplotlib as plt

def performance_measures(y_true , y_pred_pos):
  #confusion matrix
  print("Confusion matrix")
  y_pred_class = y_pred_pos > 0.5
  cm = confusion_matrix(y_true, y_pred_class)
  print(cm)
  tn, fp, fn, tp = cm.ravel()
  
  #recall score
  print("Recall score = " , recall_score(y_true, y_pred_class))

  #precision score
  print("Precision score = " , precision_score(y_true, y_pred_class))

  #beta scores
  print("F1 score =",fbeta_score(y_true, y_pred_class, 1))
  print("F2 score =", fbeta_score(y_true, y_pred_class, 2))

  #cohen-capa
  print("Cohen capa score = " , cohen_kappa_score(y_true, y_pred_class))

  # #roc-curve
  # fig, ax = plt.subplots()
  # plot_roc(y_true, y_pred_pos, ax=ax)

  #log-loss
  print("Log - loss = ", log_loss(y_true, y_pred_pos))

  #roc_auc
  print("Roc_auc score=", roc_auc_score(y_true, y_pred_pos))


In [ ]:
from keras.layers import Bidirectional
def train_lstm(is_bidirectional=False):
    tb_callback = TensorBoard(log_dir='./logs', embeddings_freq=1)

    model = Sequential()
    model.add(Embedding(num_words, 32, input_length=max_log_length))
    model.add(Dropout(0.5))
    if is_bidirectional==True:
      model.add(Bidirectional(LSTM(64, recurrent_dropout=0.5)))
    else:
      model.add(LSTM(64, recurrent_dropout=0.5))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    print(model.summary())
    model.fit(X_train, Y_train, validation_split=0.25, epochs=3, batch_size=128)

    # return model

    # Evaluate model
    score, acc = model.evaluate(X_test, Y_test, verbose=1, batch_size=128)

    print("Model Accuracy: {:0.2f}%".format(acc * 100))
    
    pred = model.predict(X_test)
    num_false_positives = 0
    num_dealt_with=0
    for i in range(len(X_test)):
      if Y_test[i]==0 and pred[i]>=0.50:
        num_false_positives+=1
        if pred[i]<=0.60:
          num_dealt_with+=1

    print("Number of false positives = " , num_false_positives)
    print("Number of false positives that are solved with the hueristic method" , num_dealt_with)
    if num_false_positives!=0:
      print("Percentage dealth with = " , (num_dealt_with*1.0/num_false_positives)*100)

    performance_measures(Y_test ,pred)
    return model , Y_test , pred




In [ ]:
#normal
lstm_model , Y_test , pred = train_lstm(False)


Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 1024, 32)          2016      
_________________________________________________________________
dropout_16 (Dropout)         (None, 1024, 32)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 64)                24832     
_________________________________________________________________
dropout_17 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 65        
Total params: 26,913
Trainable params: 26,913
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
118/118 [==============================] - 438s 4s/step - loss: 0.6764 - accuracy: 0.6048 - val_loss:

In [ ]:
# Bidirectional
lstm_model,Y_test , pred = train_lstm(True)

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 1024, 32)          2016      
_________________________________________________________________
dropout_18 (Dropout)         (None, 1024, 32)          0         
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               49664     
_________________________________________________________________
dropout_19 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 129       
Total params: 51,809
Trainable params: 51,809
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
118/118 [==============================] - 880s 7s/step - loss: 0.5169 - accuracy: 0.7277 - val_loss:

In [ ]:
ydd =[]
ddd= []

def train_vanilla_RNN(is_bidirectional=False):
    tb_callback = TensorBoard(log_dir='./logs', embeddings_freq=1)

    model = Sequential()
    model.add(Embedding(num_words, 32, input_length=max_log_length))
    model.add(Dropout(0.5))
    if is_bidirectional==True:
      model.add(Bidirectional(SimpleRNN(10, recurrent_dropout=0.5)))
    else:
      model.add(SimpleRNN(10, recurrent_dropout=0.5))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    opt = keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    print(model.summary())
    model.fit(X_train, Y_train, validation_split=0.25, epochs=3, batch_size=128)

    # return model

    # Evaluate model
    score, acc = model.evaluate(X_test, Y_test, verbose=1, batch_size=128)

    print("Model Accuracy: {:0.2f}%".format(acc * 100))
    # # Save model
    # model.save_weights('securitai-lstm-weights.h5')
    # model.save('securitai-lstm-model.h5')
    # with open('securitai-lstm-model.json', 'w') as outfile:
    #     outfile.write(model.to_json())
    # return model
    pred = model.predict(X_test)
    num_false_positives = 0
    num_dealt_with=0
    for i in range(len(X_test)):
      if Y_test[i]==0 and pred[i]>=0.50:
        num_false_positives+=1
        if pred[i]<=0.60:
          num_dealt_with+=1
    
    # performance_measures(Y_test ,pred)
    # print("Number of false positives = " , num_false_positives)
    # print("Number of false positives that are solved with the hueristic method" , num_dealt_with)
    # print("Percentage dealth with = " , (num_dealt_with*1.0/num_false_positives)*100)

    performance_measures(Y_test ,pred)
    return model



In [ ]:
# Normal
vanilla_RNN_model, Y_test , pred = train_vanilla_RNN(False)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 1024, 32)          2016      
_________________________________________________________________
dropout_8 (Dropout)          (None, 1024, 32)          0         
_________________________________________________________________
simple_rnn_4 (SimpleRNN)     (None, 10)                430       
_________________________________________________________________
dropout_9 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 11        
Total params: 2,457
Trainable params: 2,457
Non-trainable params: 0
_________________________________________________________________
None
53/53 [==============================] - 4s 71ms/step - loss: 0.7000 - accuracy: 0.4676
Model Accuracy: 46.76%


In [ ]:
# Bidrectional
vanilla_RNN_model = train_vanilla_RNN(True)

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 1024, 32)          2016      
_________________________________________________________________
dropout_14 (Dropout)         (None, 1024, 32)          0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 20)                860       
_________________________________________________________________
dropout_15 (Dropout)         (None, 20)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 21        
Total params: 2,897
Trainable params: 2,897
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
118/118 [==============================] - 234s 2s/step - loss: 0.7774 - accuracy: 0.4929 - val_loss: 0

In [ ]:
performance_measures(Y_test,pred)

Confusion matrix
[[ 550 2806]
 [ 758 2580]]
Recall score =  0.7729179149191132
Precision score =  0.4790196806535462
F1 score = 0.591471801925722
F2 score = 0.6884406019852706
Cohen capa score =  -0.06309269603841461
Log - loss =  0.7000138760904144
Roc_auc score= 0.5449097723258951


In [ ]:
print(performance_measures(Y_test,pred))

Confusion matrix
[[ 550 2806]
 [ 758 2580]]
Recall score =  0.7729179149191132
Precision score =  0.4790196806535462
F1 score = 0.591471801925722
F2 score = 0.6884406019852706
Cohen capa score =  -0.06309269603841461
Log - loss =  0.7000138760904144
Roc_auc score= 0.5449097723258951
None


In [ ]:
def train_GRU(is_bidirectional=False):
    tb_callback = TensorBoard(log_dir='./logs', embeddings_freq=1)

    tb_callback = TensorBoard(log_dir='./logs', embeddings_freq=1)

    model = Sequential()
    model.add(Embedding(num_words, 32, input_length=max_log_length))
    model.add(Dropout(0.5))
    if is_bidirectional==True:
      model.add(Bidirectional(GRU(64, recurrent_dropout=0.5)))
    else:
      model.add(GRU(64, recurrent_dropout=0.5))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    print(model.summary())
    model.fit(X_train, Y_train, validation_split=0.25, epochs=1, batch_size=128)

    # return model

    # Evaluate model
    score, acc = model.evaluate(X_test, Y_test, verbose=1, batch_size=128)

    print("Model Accuracy: {:0.2f}%".format(acc * 100))
    # # Save model
    # model.save_weights('securitai-lstm-weights.h5')
    # model.save('securitai-lstm-model.h5')
    # with open('securitai-lstm-model.json', 'w') as outfile:
    #     outfile.write(model.to_json())
    # return model
    pred = model.predict(X_test)
    # return model  , pred
    num_false_positives = 0
    num_dealt_with=0
    for i in range(len(X_test)):
      if Y_test[i]==0 and pred[i]>=0.50:
        num_false_positives+=1
        if pred[i]<=0.60:
          num_dealt_with+=1
    
    performance_measures(Y_test, pred)
    

    print("Number of false positives = " , num_false_positives)
    print("Number of false positives that are solved with the hueristic method" , num_dealt_with)
    print("Percentage dealth with = " , (num_dealt_with*1.0/num_false_positives)*100)

    return model , pred


In [ ]:
#normal
gru_model , pred = train_GRU(False)

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 1024, 32)          2016      
_________________________________________________________________
dropout_14 (Dropout)         (None, 1024, 32)          0         
_________________________________________________________________
gru_2 (GRU)                  (None, 64)                18816     
_________________________________________________________________
dropout_15 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 65        
Total params: 20,897
Trainable params: 20,897
Non-trainable params: 0
_________________________________________________________________
None
53/53 [==============================] - 16s 302ms/step - loss: 0.5983 - accuracy: 0.6243
Model Accuracy: 62.43

In [ ]:
#bidrectional
gru_model = train_GRU(True)

NameError: ignored